## MSCA 32018 1 Natural Language Processing and Cognitive Computing

### Assignment 5

**Author: Targoon Siripanichpong**

You have been provided with a pickle file, containing the 100 news articles about Caterpillar.  Identify what companies  are mentioned most frequently in the news along with Caterpillar. 

pd.read_pickle(directory+'news_cat.pkl')
Discard non-English results
Identify what companies are mentioned most frequently along with Caterpillar (in both title and the body of the article)
Show a table or chart with your top-20 companies (sorted in the descending order)
Rules and requirements:

Your final output and the code should be contained within Jupyter Notebook (ipynb)

#### Import Data

In [1]:
import pickle

In [2]:
cat_news = pickle.load( open( "../Data/news_cat.pkl", "rb" ) )

In [3]:
type(cat_news)

pandas.core.frame.DataFrame

In [4]:
cat_news.head()

,crawled,language,text,title
0,2018-01-30T23:03:51.004+02:00,english,by Abhishek K Global Telehandler Market 2023 D...,Global Telehandler Market 2023 Demand by Segme...
1,2018-01-30T23:06:46.024+02:00,english,favorite this post 2014 Caterpillar 314E LCR h...,2014 Caterpillar 314E LCR
2,2018-01-30T23:18:35.023+02:00,english,By: MAX NISEN The Amazon health care threat ha...,"Amazon, Berkshire, JPMorgan health announcemen..."
3,2018-01-30T23:20:54.012+02:00,english,QR Code Link to This Post MONTHLY PUBLIC AUCTI...,2005 Caterpillar CB534D Tandem Vibratory Rolle...
4,2018-01-30T23:28:30.000+02:00,english,QR Code Link to This Post 2007 CATERPILLAR D4G...,2007 CATERPILLAR D4G LGP CAB SCREEN/SWEEPS - O...


In [5]:
len(cat_news)

100

In [6]:
cat_news_en = cat_news[cat_news['language'] == 'english']

In [7]:
len(cat_news_en)

100

#### Use Watson NLU API to perform Entity Recognition

In [50]:
import sys
import os
import json
import re
import pandas as pd
import ibm_watson
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions, ConceptsOptions, RelationsOptions, SentimentOptions, SemanticRolesOptions, EmotionOptions, CategoriesOptions
import time

In [55]:
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2018-11-16',iam_apikey="Jl-4Ls8SByMZIKX_B72ykHNQwR5PbK-q1pkvSEEWLt60",
        url="https://gateway.watsonplatform.net/natural-language-understanding/api")

In [56]:
def get_company_list(text):
    response = natural_language_understanding.analyze(
                language = "en", text = text,
                features = Features(
                            entities = EntitiesOptions(sentiment = False, emotion = False)
                            #keywords = KeywordsOptions(sentiment = True, emotion = True),
                            #concepts = ConceptsOptions(limit = 50),
                            #relations = RelationsOptions(),
                            #sentiment = SentimentOptions(),
                            #semantic_roles = SemanticRolesOptions(entities = True, keywords = True),
                            #emotion = EmotionOptions(),
                            #categories = CategoriesOptions()
                )).get_result()
    company_list = [entity['text'] for entity in response['entities'] if entity['type'] == 'Company'] #extract only companies
    time.sleep(0.1) #Avoid throttling limit
    return(company_list)

In [57]:
agg_company_list_text = [get_company_list(text) for text in cat_news_en['text']]
agg_company_list_title = [get_company_list(text) for text in cat_news_en['title']]

In [60]:
agg_company_list_text

[['Telehandler Market Product',
  'Abhishek K Global Telehandler',
  'Telehandler Price',
  'Telehandler',
  'Wacker Neuson',
  'Key Companies List',
  'Doosan Infracore',
  'Terex',
  'JCB',
  'Region Operation',
  'Caterpillar'],
 [],
 [],
 ['Service Trucks', 'Passenger Cars'],
 ['Expert Heavy Equipment Inc.', 'komatsu'],
 ['Caterpillar Inc.',
  'Caterpillar',
  '2,191 Caterpillar Inc.',
  'Elite Wealth Management Inc',
  'Acrospire Investment Management LLC',
  'Quadrant Capital Group LLC',
  'Wealth Management Inc.',
  'BMO Capital Markets',
  'Harvest Fund Management Co',
  'Motco',
  'Goldman Sachs Group',
  'Citigroup',
  'NYSE',
  'Blue Chip Partners Inc.',
  '5,214',
  'Construction Industries',
  'OTR Global',
  'Resource Industries',
  'Energy & Transportation',
  'The Lincolnian Online'],
 [],
 ['Insomniac', 'Caterpillar', 'Tiesto'],
 ['Goldman Sachs',
  'Wal-Mart',
  'Apple',
  'WMT',
  'McDonald',
  'Cisco',
  'Intel',
  'Pfizer',
  'Home Depot',
  'JPMorgan',
  'PFE',
  

In [67]:
#Flatten the list
from itertools import chain
agg_company_list_text_flat = list(chain(*agg_company_list_text))
agg_company_list_title_flat = list(chain(*agg_company_list_title))

In [68]:
company_list = agg_company_list_text_flat + agg_company_list_title_flat

In [84]:
#Remove Caterpillar and similar names off the list
import re
pattern = re.compile(r'\bCaterpillar\b')

company_list = [(text, not bool(pattern.search(text))) for text in company_list]
company_list = [text for text, tf in company_list if tf]

In [89]:
import collections
counter=collections.Counter(company_list)
print(counter.most_common(20))

[('NYSE', 26), ('Resource Industries', 19), ('Construction Industries', 18), ('Energy & Transportation', 16), ('Citigroup', 13), ('Apple', 8), ('Vetr', 8), ('DIESEL', 8), ('JPMorgan Chase', 7), ('Bank of America', 7), ('The Lincolnian Online', 6), ('Credit Suisse Group', 6), ('Amazon', 6), ('Reuters', 6), ('Acrospire Investment Management LLC', 5), ('Motco', 5), ('Goldman Sachs Group', 5), ('5,214', 5), ('JPMorgan', 5), ('Deutsche Bank', 5)]
